<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_Caching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding LangChain Caching

In this notebook, we will see:
1. How LangChain framework uses caching mechanism to improve LLM interaction efficiency.
2. The caching algorithms of 2 different underlying storages, In-Memory and SQLite.

Hope it will help you understand if and when you should use CACHE.

In [1]:
!pip install langchain openai --quiet --upgrade

In [2]:
import os
os.environ['OPENAI_API_KEY'] = 'your openai api key'

## Get your ChatOpenAI instance ready

In [3]:
import langchain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()

1. In Memory Cache

In [44]:
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

Ask a question and measure how long it takes for LLM to respond.

In [45]:
%%time

llm.predict("What is OpenAI?")

CPU times: user 25 ms, sys: 6.4 ms, total: 31.4 ms
Wall time: 4.54 s


"OpenAI is an artificial intelligence research laboratory and company that aims to ensure that artificial general intelligence (AGI) benefits all of humanity. It was founded in December 2015 as a non-profit organization but later transformed into a for-profit company called OpenAI LP in 2019. OpenAI conducts research in various fields of AI, develops cutting-edge technologies, and publishes most of its AI research findings. The organization's mission is to ensure that AGI is developed safely, is aligned with human values, and is used for the benefit of all individuals."

In [57]:
# First element of the tuple
list(langchain.llm_cache._cache.keys())[0][0]

'[{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "HumanMessage"], "kwargs": {"content": "What is OpenAI?"}}]'

In [58]:
# Second element of the tuple
list(langchain.llm_cache._cache.keys())[0][1]

'{"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}}}---[(\'stop\', None)]'

Ask same question again and see the quicker response.

In [ ]:
%%time

llm.predict("What is OpenAI?")

2. SQLite as Cache

In [6]:
!rm -f .cache.db

In [18]:
from langchain.cache import SQLiteCache
langchain.llm_cache = SQLiteCache(database_path=".cache.db")

Ask the same question twice and measure the performance difference

In [19]:
%%time

llm.predict("What is OpenAI?")

CPU times: user 4.25 ms, sys: 980 µs, total: 5.23 ms
Wall time: 4.97 ms


'OpenAI is an artificial intelligence research laboratory and company. It was founded in December 2015 with the goal of developing and promoting friendly AI for the benefit of all humanity. OpenAI conducts cutting-edge research in various areas of AI and aims to ensure that artificial general intelligence (AGI) benefits everyone and is used responsibly. They work on advancing AI technology, publishing most of their AI research, and providing public goods to help society navigate the path to AGI. OpenAI also develops and deploys AI models and systems, such as the language model GPT-3, to showcase the capabilities and potential applications of AI.'

In [42]:
%%time

llm.predict("What is OpenAI?")

CPU times: user 39.3 ms, sys: 9.16 ms, total: 48.5 ms
Wall time: 4.84 s


'OpenAI is an artificial intelligence research lab and company founded in December 2015. Its mission is to ensure that artificial general intelligence (AGI) benefits all of humanity. OpenAI conducts research to develop safe and beneficial AI technologies and also aims to promote the widespread adoption of such technologies for societal benefit. The organization has made significant contributions to the field of AI, particularly in areas such as natural language processing, reinforcement learning, and robotics. OpenAI also develops and maintains various open-source AI tools and frameworks to facilitate the development and deployment of AI applications.'

Add some space in the sentence and ask again

In [ ]:
%%time

llm.predict("What is  OpenAI?")

In [39]:
import sqlalchemy
from sqlalchemy import create_engine, text
engine = create_engine("sqlite:///.cache.db")

**Why does the extra space cause the cache miss??**

In [43]:
with engine.connect() as connection:

    rs = connection.exec_driver_sql('select * from full_llm_cache')
    print(rs.keys())
    for row in rs:
        print(row)

RMKeyView(['prompt', 'llm', 'idx', 'response'])
('[{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "HumanMessage"], "kwargs": {"content": "What is OpenAI?"}}]', '{"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}}}---[(\'stop\', None)]', 0, '{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "ChatGeneration"], "kwargs": {"message": {"lc": 1, "type": "constructor", "id": ["lang ... (588 characters truncated) ... AI models and systems, such as the language model GPT-3, to showcase the capabilities and potential applications of AI.", "additional_kwargs": {}}}}}')
('[{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "HumanMessage"], "kwargs": {"content": "What is  OpenAI?"}}]', '{"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"openai_api_key": {"lc": 1, "type": "secret", "